# Feature Engineering

In [1]:
# Take the first n numbers from the upc as a string
# Take the upc length as a string
# Make departments all separate features and count the number of items purchased under each department
# Make broader departments such as Clothing for all Clothing
# Weekend vs Weekday indicator (1 and 0)
# Make null a binary indicator (1 and 0)
# Replace upc null with "no_upc"
# Replace finelinenumber null with "no_fineline"
# Ratio of total_scancount / num_unique_upc = avg_scancount_per_upc
# Ratio of total_scancount / num_unique_dept = avg_scancount_per_dept
# Ratio of num_unique_fineline / num_unique_dept = fineline_to_dept_ratio
# Ratio of num_unique_upc / num_unique_fineline = avg_upc_per_fineline (maybe)
# Ratio of num_unique_upc / num_unique_dept = avg_upc_per_dept

In [2]:
# Imports
import pandas as pd
import numpy as np

In [4]:
# Import data
original_data = pd.read_csv('../data-ignore/train.csv', dtype={'Upc': str})
original_data.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,60538815980,1,SHOES,8931.0
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504.0
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017.0


In [5]:
# Group by VisitNumber to get features and target (triptype)
df = original_data.groupby('VisitNumber').agg(triptype=('TripType','first'),
                              weekday=('Weekday','first'),
                              num_unique_upc=('Upc','nunique'),
                              avg_scancount=('ScanCount','mean'),
                              total_scancount=('ScanCount','sum'),
                              num_unique_dept=('DepartmentDescription','nunique'),
                              num_unique_fileline=('FinelineNumber','nunique')
                              )
df.reset_index(inplace=True)
df

,VisitNumber,triptype,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline
0,5,999,Friday,1,-1.000000,-1,1,1
1,7,30,Friday,2,1.000000,2,2,2
2,8,26,Friday,20,1.217391,28,6,16
3,9,8,Friday,3,1.000000,3,2,3
4,10,8,Friday,3,1.000000,3,2,3
...,...,...,...,...,...,...,...,...
95669,191343,25,Sunday,7,1.285714,9,3,5
95670,191344,22,Sunday,5,1.000000,5,2,3
95671,191345,39,Sunday,13,1.307692,17,8,12
95672,191346,39,Sunday,17,1.000000,17,8,16
